## Q2: Transfer Learning for Fashion-MNIST

we adapted a pretrained ResNet50 model to classify Fashion-MNIST grayscale images into 10 categories using transfer learning:
- Prepared a data pipeline (resize, convert grayscale to RGB)
- Build and train a model with a frozen backbone and a new classification head
- Fine-tune deeper layers with a lower learning rate
- Experimented with data augmentation, learning rate scheduling, and regularization


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# Load Fashion-MNIST data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Data augmentation pipeline
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])

def create_dataset(images, labels, batch_size=32, augment=False):
    ds = tf.data.Dataset.from_tensor_slices((images, labels))
    def preprocess(x, y):
        x = tf.image.resize(x[..., tf.newaxis], (224, 224))
        x = tf.repeat(x, 3, axis=-1)
        if augment:
            x = data_augmentation(x)
        return x / 255.0, y
    return ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).batch(batch_size).prefetch(tf.data.AUTOTUNE)

batch_size = 32
train_ds = create_dataset(x_train, y_train, batch_size, augment=True)
val_ds = create_dataset(x_test, y_test, batch_size)


### 1. Data Loading and Preprocessing

We load the Fashion-MNIST dataset, resize images to 224x224, convert grayscale to 3-channel RGB, and normalize pixel values. Data augmentation is included for improved generalization.


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import models

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])


### 2. Model Construction

We use ResNet50 pretrained on ImageNet as the backbone (without top layers). The backbone is frozen, and a new classification head is added for Fashion-MNIST's 10 classes.


In [ ]:
tf.keras.mixed_precision.set_global_policy('mixed_float16')

from tensorflow.keras import optimizers

# Learning rate schedule
lr_schedule = optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=1000,
    decay_rate=0.9
)

model.compile(
    optimizer=optimizers.Adam(learning_rate=lr_schedule),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_head = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds
)


### 3. Training the Classification Head

We train only the new head while keeping the backbone frozen. Mixed precision training is enabled for efficiency. Validation accuracy is recorded after each epoch.


In [ ]:
base_model.trainable = True
for layer in base_model.layers[:-16]:
    layer.trainable = False

model.compile(
    optimizer=optimizers.Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_full = model.fit(
    train_ds,
    epochs=5,
    validation_data=val_ds
)


### 4. Fine-Tuning the Backbone

We unfreeze the last 16 layers of ResNet50 and continue training with a lower learning rate to adapt higher-level features to Fashion-MNIST.


In [ ]:
test_loss, test_acc = model.evaluate(val_ds)
print(f"Final Test Accuracy: {test_acc*100:.2f}%")

import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history_head.history['val_accuracy'], label='Head Training')
plt.plot(range(len(history_head.history['val_accuracy']),
         len(history_head.history['val_accuracy']) + len(history_full.history['val_accuracy'])),
         history_full.history['val_accuracy'], label='Fine-Tuning')
plt.title('Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_head.history['val_loss'], label='Head Training')
plt.plot(range(len(history_head.history['val_loss']),
         len(history_head.history['val_loss']) + len(history_full.history['val_loss'])),
         history_full.history['val_loss'], label='Fine-Tuning')
plt.title('Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


### 5. Evaluation and Visualization

We evaluate the final model on the test set and plot validation accuracy and loss curves for both training phases.


### 6. Conclusion

We successfully adapted a pretrained ResNet50 model to classify Fashion-MNIST images using transfer learning. The approach included data augmentation, learning rate scheduling, regularization, and fine-tuning, resulting in improved validation accuracy.
